In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
from nltk.corpus import stopwords
import nltk
from gensim.models import ldamodel
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import normalize
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
import sys

import os
import plotly

import base64
import numpy as np
import pandas as pd
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from collections import Counter
from scipy.misc import imread
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from matplotlib import pyplot as plt
%matplotlib inline
plotly.offline.init_notebook_mode(connected=True)
import plotly.offline as py


from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
stop.update(['href','br'])
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
data = pd.read_excel("/Users/dsg281/Downloads/Dubai_now_log/Dubai_Now_Logs_English.xlsx")

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data = pd.DataFrame(data)

In [ ]:
for idx in range(len(data)):
    
    data.iloc[idx]['Questions'] = [word for word in tokenizer.tokenize(data.iloc[idx]['Questions'].lower()) if word not in stop]

In [ ]:
data['Questions'] = data['Questions'].apply(lambda x: [item for item in x if item not in stop])

In [ ]:
#Number of Topics

In [ ]:
train_ = [value[0] for value in data.iloc[0:].values]
num_topics = 5

In [ ]:
bigram = gensim.models.Phrases(train_)
texts = [bigram[line] for line in train_]

In [ ]:
id2word = gensim.corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]
lda = ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics,chunksize=100,alpha='auto')

In [ ]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 10);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict)

In [ ]:
get_lda_topics(lda, num_topics)


In [ ]:
#Try2

In [ ]:
data = pd.read_excel("/Users/dsg281/Downloads/Dubai_now_log/Dubai_Now_Logs_English.xlsx")

In [ ]:
def display_topics(model, feature_names, no_top_words):
    
    for topic_idx, topic in enumerate(model.components_):
        print("Topic:", (topic_idx))
        print(" ".join([feature_names[i]
        for i in topic.argsort()[:-no_top_words - 1:-1]]))


def tfidf_vectorizer(documents,total_features):

    #  TFIDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=total_features,ngram_range=(2, 3),stop_words='english',analyzer='word',lowercase=True)
    tfidf = tfidf_vectorizer.fit_transform(documents)
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    return tfidf_vectorizer,tfidf,tfidf_feature_names

def count_vectorizer(documents,total_features):

    #  Count Vectorizer
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, ngram_range=(2, 3),max_features=total_features, stop_words='english')
    tf = tf_vectorizer.fit_transform(documents)
    tf_feature_names = tf_vectorizer.get_feature_names()
    return tf_vectorizer,tf,tf_feature_names

In [ ]:
total_features = 200
num_topic = 5

In [ ]:
tfidf_vectorizer, tfidf, tfidf_feature_names = tfidf_vectorizer(data['Questions'],total_features)

In [ ]:
tf_vectorizer, tf, tf_feature_names = count_vectorizer(data['Questions'],total_features)

In [ ]:
tfidf_feature_names[:5]

In [ ]:
model_lda = LatentDirichletAllocation(n_topics=num_topic, max_iter=50, learning_method='online', learning_offset=50.,random_state=0).fit(tfidf)
no_top_words = 5

In [ ]:
display_topics(model_lda, tfidf_feature_names, no_top_words)


In [ ]:
data_op = pyLDAvis.sklearn.prepare(model_lda,tfidf,tfidf_vectorizer)


In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.display(data_op)

In [ ]:
#Try3 - Final Select NMF

In [ ]:
vectorizer = CountVectorizer(analyzer='word', max_features=50,ngram_range=(1,2),stop_words='english')
x_counts = vectorizer.fit_transform(data['Questions'])


In [ ]:
transformer = TfidfTransformer(smooth_idf=False)
x_tfidf = transformer.fit_transform(x_counts)

In [ ]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)


In [ ]:
model = NMF(n_components=num_topics, init='nndsvd');


In [ ]:
model.fit(xtfidf_norm)


In [ ]:
def get_nmf_topics(model, n_top_words):
    
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-3 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [ ]:
get_nmf_topics(model, 3)
